<a href="https://colab.research.google.com/github/marianoogimenez/google-asl-project/blob/master/PARTE_00_recomendacion_de_contenido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PARTE 00 --> ENTRENO 400K DE NOTAS

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import matplotlib.pyplot  as plt
#import pattern.es 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import NMF
import unidecode
import gensim
from nltk import SnowballStemmer
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
import pickle

In [3]:
df = pd.read_csv("C:\\Users\\ldalera.AGEA\\Documents\\Google\\Recomendacion\\dataset\\salida.txt",sep="\\|-\\|",header=None,encoding="utf-8")

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


FileNotFoundError: ignored

In [0]:
def limpiador_cuerpo(x):
    x= x.strip()
    x= x.lower()
    x= re.sub("&#xe1;","á",x)
    x= re.sub("&#xe9;","é",x)
    x= re.sub("&#xed;","í",x)
    x= re.sub("&#xf3;","ó",x)
    x= re.sub("&#xfa;","ú",x)
    x= re.sub('<.*?>', '',x)
    x= re.sub(pattern="&quot;",repl="",string=x)
    x= re.sub("&.*?;"," ",x)
    x= re.sub(pattern="[^a-zA-Záéíóúñ \n]",repl="",string=x)
    return x


def sacar_stop(tokens,idioma="spanish",min_len = 3,stemizar= False):
    stopwords = nltk.corpus.stopwords.words(idioma)
    tok_filt = [token for token in tokens if token not in stopwords and len(token)>=min_len]
    #tok_filt = [token for token in tok_filt if token not in nltk.corpus.stopwords.words("english")]
    if stemizar:
        tok_filt = [self.stem.stem(token) for token in tok_filt]
    return tok_filt

In [0]:
df.columns = ['content_id', 'body_raw', 'title', 'summary', 'related_1', 'related_2',
       'related_3', 'section_1', 'section_2', 'section_3', 'publish_date',
       'subtitle', 'tag1', 'tag2', 'tag3', 'tag4', 'url']

df.head()

In [0]:
df = df.dropna(subset=["body_raw"])

In [0]:
notas_tokenizadas = [sacar_stop(limpiador_cuerpo(x).split(" ")) for x in df.body_raw]

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [0]:
tagged_data = [TaggedDocument(_d, tags=[str(i)]) for i, _d in enumerate(notas_tokenizadas)]

In [0]:
max_epochs = 40
vec_size = 50

modeld2v = Doc2Vec(size=vec_size,
                min_count=2,
                dm =1)
  
modeld2v.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    modeld2v.train(tagged_data,
                total_examples=modeld2v.corpus_count,
                epochs=modeld2v.iter)
    # decrease the learning rate
    modeld2v.alpha -= 0.0002
    # fix the learning rate, no decay
    modeld2v.min_alpha = modeld2v.alpha

modeld2v.save("d2v.model")
print("Model Saved")